In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
import requests
load_dotenv()

True

In [2]:
# Tool Creation
@tool
def multiply(a: int, b:int) -> int:
    """Given 2 numbers a and b and returns their product"""
    return a * b

In [3]:
print(multiply.invoke({"a":3,"b":4}))

12


In [4]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b and returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
# Tool Binding
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite-preview-06-17",
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0.5
)

In [6]:
llm_with_tool = llm.bind_tools([multiply])

In [7]:
llm_with_tool

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite-preview-06-17', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x79f4fff3ea50>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Given 2 numbers a and b and returns their product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [8]:
llm_with_tool.invoke("Hi How are you?")

AIMessage(content="I'm doing well. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--5d4c8bfb-e0ce-4ec5-8061-d6a68edde4dd-0', usage_metadata={'input_tokens': 58, 'output_tokens': 13, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}})

In [9]:
query = HumanMessage("can you multiply 4 with 10?")

In [10]:
messages = [query]

In [11]:
messages

[HumanMessage(content='can you multiply 4 with 10?', additional_kwargs={}, response_metadata={})]

In [13]:
result = llm_with_tool.invoke(messages)

In [14]:
messages.append(result)

In [15]:
# Tool Execution
# ToolMessage is a special message when we execute the tool with the help of toolcall
tool_result = multiply.invoke(result.tool_calls[0])

In [16]:
messages.append(tool_result)

In [17]:
messages

[HumanMessage(content='can you multiply 4 with 10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--712bd458-3f41-431d-88de-25099fd0cc27-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4.0, 'b': 10.0}, 'id': '313a1365-0350-42b4-8ca7-48d6436d8a13', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19, 'total_tokens': 82, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='40', name='multiply', tool_call_id='313a1365-0350-42b4-8ca7-48d6436d8a13')]

In [18]:
llm_with_tool.invoke(messages).content

'40'